In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from helper import l2_norm, poisson_2d_jacobi, poisson_solution

In [20]:
# Set parameters.
nx, ny = 41, 41  # number of points in each direction
L = Lx = Ly = 1.0  # length of the square cavity
dx = L / (nx - 1)  # grid spacing in the x direction
dy = L / (ny - 1)  # grid spacing in the y direction

In [53]:
xmin, xmax = 0.0, 1.0
ymin, ymax = -0.5, 0.5
Lx = xmax - xmin
Ly = ymax - ymin

x = np.linspace(xmin, xmax, nx)
y = np.linspace(ymin, ymax, ny)
X, Y = np.meshgrid(x, y)

#Source Term
b = -2*(np.pi/2)**2 * np.sin(np.pi * X / L_x) * np.cos(np.pi*Y/L_y)

#Initial Guess
p0 = np.zeros((ny, nx))

#Exact Solution of Poisson System
p_exact = poisson_solution(x, y, Lx, Ly)

In [54]:
def poisson_2d_steepest(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    
    def A(p):
        """Compute Laplacian of p"""
        return (-4.0 * p[1:-1, 1:-1] + p[:-2, 1:-1] + p[2:,1:-1]
                        + p[1:-1,:-2]+ p[1:-1, 2:]) / dx**2
    
    p = p0.copy()
    rk = np.zeros_like(p)
    Ar = np.zeros_like(p)
    diff = rtol + 1
    ite = 0
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        #Compute residual rk
        rk[1:-1,1:-1] = b[1:-1, 1:-1] - A(p)
        #Compute step size alpha
        Ar[1:-1,1:-1] = A(rk)
        alpha = np.sum(rk * rk) / np.sum(rk * Ar)
        
        #Update Solution
        p = pk + alpha*rk
        
        
        #Compute Relative Difference
        diff = l2_norm(p, pk)
        #Increment iteration index
        ite+=1 #pretty lame you can't do i++ in python :(
    
    
    return p, ite, diff

In [55]:
p_1, ite_1, diff_1 = poisson_2d_steepest(p0, b, dx, dy, rtol=1e-10)
print(ite_1, diff_1)

2 1.4379144656378503e-16


In [60]:
def poisson_2d_conjugate_gradient(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    
    def A(p):
        """Compute Laplacian of p"""
        return (-4.0 * p[1:-1, 1:-1] + p[:-2, 1:-1] + p[2:,1:-1]
                        + p[1:-1,:-2]+ p[1:-1, 2:]) / dx**2
    
    p = p0.copy()
    r = np.zeros_like(p)
    Ad = np.zeros_like(p)
    diff = rtol + 1
    ite = 0
    r[1:-1,1:-1] = b[1:-1,1:-1] - A(p)
    
    d = r.copy()
    
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        rk = r.copy()
        dk = d.copy()
        
        Ad[1:-1, 1:-1] = A(dk)
        #Compute step size alpha
        alpha = np.sum(rk * rk) / np.sum(Ad * dk)
        
        #Update Solution
        p = pk + alpha*rk
        
        r = rk - alpha*Ad
        
        beta = np.sum(r*r) / np.sum(rk*rk)
        
        d = r + beta*dk
        
        
        #Compute Relative Difference
        diff = l2_norm(p, pk)
        #Increment iteration index
        ite+=1 #pretty lame you can't do i++ in python :(
    
    
    return p, ite, diff

In [61]:
p_2, ite_2, diff_2 = poisson_2d_conjugate_gradient(p0, b, dx, dy,
                                                 maxiter=20000,
                                                 rtol=1e-10)
print('Method of conjugate gradients: {} iterations '.format(ite_2) +
      'to reach a relative difference of {}'.format(diff_2))

Method of conjugate gradients: 2 iterations to reach a relative difference of 1.449687382823206e-16


In [68]:
# Modify the source term of the Poisson system.
b2 = (np.sin(np.pi * X / Lx) *
     np.cos(np.pi * Y / Ly) +
     np.sin(6.0 * np.pi * X / Lx) *
     np.sin(6.0 * np.pi * Y / Ly))

p_steep2, ite_steep2, diff_steep2 = poisson_2d_steepest(p0, b2, dx, dy, rtol=1e-10)
print("Steepest decent took ", ite_steep2, "iterations to reach a residual of:", diff_steep2)

p_cg2, ite_cg2, diff_cg2 = poisson_2d_conjugate_gradient(p0, b2, dx, dy, rtol=1e-10)
print("Conjugate Gradient took ", ite_cg2, "iterations to reach a residual of:", diff_cg2)

Steepest decent took  2884 iterations to reach a residual of: 9.967120827790515e-11
Conjugate Gradient took  3 iterations to reach a residual of: 5.40247261090058e-15
